# Call Rest Api
AutoAIでデプロイしたWebサービスを呼び出すためのサンプルコード

In [1]:
# 変数定義
# bank marketing

# テスト用入力ファイル名
infile = 'bank-test.csv'

# 出力ファイル名
outfile = 'bank-result.xlsx'

# 目的変数名 (入力ファイルに目的変数が含まれている場合)
y_name = 'y'

In [48]:
# 変数定義

# テスト用入力ファイル名
infile = 'iris-test.csv'

# 出力ファイル名
outfule= 'iris-result.xlsx'

# 目的変数名 (入力ファイルに目的変数が含まれている場合)
# 目的変数の列がない場合は 'dummy' などダミーの列名を指定
y_name = 'class'

In [2]:
# Watson ML 資格情報
wml_credentials = {
  "apikey": "xxxx",
  "iam_apikey_description": "xxxx",
  "iam_apikey_name": "wdp-writer",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "xxxx",
  "instance_id": "xxxx",
  "url": "https://us-south.ml.cloud.ibm.com"
}

In [3]:
# scoring URLの設定
# bank marketing
scoring_url = "https://us-south.ml.cloud.ibm.com/v4/deployments/d933f6bf-1022-43d2-b37c-a604d021b84d/predictions"

In [49]:
# iris-model
scoring_url = 'https://us-south.ml.cloud.ibm.com/v4/deployments/ef44ebef-f545-45c5-a2e5-643232f98cad/predictions'

In [50]:
# 必要ライブラリの導入
!pip install japanize-matplotlib | tail -n 1

# 必要ライブラリのimport
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 日本語化対応
import japanize_matplotlib #日本語化matplotlib

### 必要ライブラリのimport

In [51]:
import urllib3, requests, json
import urllib.request

### Watson ML呼出し準備

In [ ]:
# トークン取得
apikey = wml_credentials["apikey"]

# Get an IAM token from IBM Cloud
url     = "https://iam.bluemix.net/oidc/token"
headers = { "Content-Type" : "application/x-www-form-urlencoded" }
data    = "apikey=" + apikey + "&grant_type=urn:ibm:params:oauth:grant-type:apikey"
IBM_cloud_IAM_uid = "bx"
IBM_cloud_IAM_pwd = "bx"
response  = requests.post( url, headers=headers, data=data, 
        auth=( IBM_cloud_IAM_uid, IBM_cloud_IAM_pwd ) )
iam_token = response.json()["access_token"]

print('iam_token = ', iam_token)

In [53]:
# Header の組立て
header_token = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + iam_token}
ml_instance_id = wml_credentials["instance_id"]
header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + iam_token, 'ML-Instance-ID': ml_instance_id}

### テスト用CSV読み込み

In [54]:
# データフレームに読み込み
df = pd.read_csv(infile)

In [55]:
df1 = df.copy()
# np.nan値をNoneに置き換え
df1 = df1.where((pd.notnull(df1)), None)

In [56]:
# 内容確認
display(df1.head())
print(df1.shape)

,class,x1,x2,x3,x4
0,1,6.1,3.0,4.6,1.4
1,1,6.1,2.9,4.7,1.4
2,2,6.3,2.9,5.6,1.8
3,0,4.6,3.4,1.4,0.3
4,1,5.2,2.7,3.9,1.4


(45, 5)


###  目的変数列の削除

In [57]:
df2 = df1.copy()

In [58]:
if y_name in df2.columns:
    y = df[y_name].values
    df2 = df2.drop(y_name, axis=1)

### API呼出し用入力データ配列の作成

In [59]:
# DataFrameから項目名Listの生成
fields = df2.columns.tolist()
print(fields)

['x1', 'x2', 'x3', 'x4']


In [60]:
# DataFrameから入力用Listの生成

# テスト用
values1 = df2.head(2).values.tolist()

# 本番用
values2 = df2.values.tolist()
print(values1)

[[6.1, 3.0, 4.6, 1.4], [6.1, 2.9, 4.7, 1.4]]


### API呼び出し(テスト)

In [61]:
# payload変数の組立て
payload = {"input_data": [{
    "fields": fields, 
    "values": values1
}]}

In [62]:
# APIの呼出し
response = requests.post(scoring_url,  json=payload, headers=header)
print(response)

<Response [200]>


In [63]:
# 戻り値のparse
res = json.loads(response.text)

pred = res['predictions'][0]
pred_values = pred['values']
pred_fields = pred['fields']
df_res = pd.DataFrame(pred_values, columns = pred_fields)
display(df_res)

,prediction,probability
0,1,"[0.006038466040442731, 0.9753126125197338, 0.0..."
1,1,"[0.006038466040442731, 0.9753126125197338, 0.0..."


### API呼び出し(本番)

In [64]:
# payload変数の組立て
payload = {"input_data": [{
    "fields": fields, 
    "values": values2
}]}

In [65]:
# APIの呼出し
response = requests.post(scoring_url,  json=payload, headers=header)
print(response)

<Response [200]>


In [66]:
# 戻り値のparse
res = json.loads(response.text)

pred = res['predictions'][0]
pred_values = pred['values']
pred_fields = pred['fields']
df_res = pd.DataFrame(pred_values, columns = pred_fields)
display(df_res.head())

,prediction,probability
0,1,"[0.006038466040442731, 0.9753126125197338, 0.0..."
1,1,"[0.006038466040442731, 0.9753126125197338, 0.0..."
2,2,"[0.005305501325490514, 0.009639628435876606, 0..."
3,0,"[0.9909271419523884, 0.0033313586298476298, 0...."
4,1,"[0.006038466040442731, 0.9753126125197338, 0.0..."


In [67]:
w = df_res.probability
df_res = df_res.drop('probability', axis=1)
for i in range(len(w[0])):
    key = 'probability%d' % i
    df_res[key] = w.map(lambda x: x[i])
display(df_res.head())

,prediction,probability0,probability1,probability2
0,1,0.006038,0.975313,0.018649
1,1,0.006038,0.975313,0.018649
2,2,0.005306,0.009640,0.985055
3,0,0.990927,0.003331,0.005741
4,1,0.006038,0.975313,0.018649


In [68]:
df_result = pd.concat([df, df_res], axis=1)

In [69]:
display(df_result.head())

,class,x1,x2,x3,x4,prediction,probability0,probability1,probability2
0,1,6.1,3.0,4.6,1.4,1,0.006038,0.975313,0.018649
1,1,6.1,2.9,4.7,1.4,1,0.006038,0.975313,0.018649
2,2,6.3,2.9,5.6,1.8,2,0.005306,0.009640,0.985055
3,0,4.6,3.4,1.4,0.3,0,0.990927,0.003331,0.005741
4,1,5.2,2.7,3.9,1.4,1,0.006038,0.975313,0.018649


In [70]:
# Excelに書き出し
df_result.to_excel(outfile, index=False)